# Imports

In [ ]:
import itertools
import os
import sys

import dotenv
import huggingface_hub
import numpy as np
import pandas as pd

from IPython.display import display

dotenv.load_dotenv()
sys.path.append("../../src")
import dataframe_utils
import huggingface_utils
import GiveMeSomeCredit

# Variables

In [ ]:
# MODEL_ID = "meta-llama/Meta-Llama-3-8B-Instruct"
# MODEL_ID = "mistralai/Mistral-7B-v0.1"
MODEL_ID = "openai/gpt-oss-120b"

MODEL_URL = f"https://huggingface.co/{MODEL_ID}"
print(f"Model URL: {MODEL_URL}")

API_URL = f"https://api-inference.huggingface.co/models/{MODEL_ID}"
print(f"API URL: {API_URL}")

HF_TOKEN = os.environ.get("HUGGINGFACEHUB_API_TOKEN")
if HF_TOKEN is None:
    raise RuntimeError("Please set HUGGINGFACEHUB_API_TOKEN in your environment variables.")
print(f"Huggingface Token: {HF_TOKEN[:3]}...{HF_TOKEN[-4:]}")

TASK = "conversational"
if not huggingface_utils.model_supports_task(MODEL_ID, TASK):
    raise RuntimeError(f"Model '{MODEL_ID}' does not support '{TASK}'.")

In [ ]:
DESCRIPTION_COLUMN = ("Input", "Description")
QUESTION_COLUMN = ("Input", "Question")
FOLLOW_UP_COLUMN = ("Input", "Question")
ANSWER_COLUMN = (MODEL_ID, "LLM Answer")
REASONING_COLUMN = (MODEL_ID, "LLM Reasoning")

# Load the Dataset

This section loads the validation data into a DataFrame and displays its basic information.


In [ ]:
classification_df = GiveMeSomeCredit.load_dataframe(
    "classification_prompts.csv", header=[0,1]
).astype("string")

with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(
        dataframe_utils.describe_df(classification_df)
    )

with pd.option_context("display.max_colwidth", None):
    display(
        classification_df.head(5).style.set_properties(**{"text-align": "left"})
    )

# Helper Functions

## `get_llm_classification`

In [ ]:
def get_llm_response(prompt, model_id, token):
    client = huggingface_hub.InferenceClient(model=model_id, token=token)
    messages = [{"role": "user", "content": prompt}]
    
    response = client.chat_completion(messages=messages)
    if response and response.choices and hasattr(response.choices[0], "message"):
        return response.choices[0].message["content"]
    else:
        raise ValueError("No valid content returned from model.")

## `get_batch_llm_classification`

In [ ]:
def get_batch_llm_responses(prompts, model_id, token):
    results = list()
    try:
        for ii, prompt in enumerate(prompts):
            result = get_llm_response(prompt, model_id=model_id, token=token)
            results.append(result)
    except Exception as e:
        print(f"Exception occurred after collecting {len(results)} results.")
        raise e
    return results

## `chunked`

In [ ]:
def chunked(iterable, n):
    it = iter(iterable)
    while True:
        chunk = list(itertools.islice(it, n))
        if not chunk:
            break
        yield chunk

## `classify`

In [ ]:
# from huggingface_hub.utils import HfHubHTTPError

def classify(
    df, description_column, question_column, answer_column, model_id, token, batch_size=10
):
    missing_indices = df[df[answer_column].isna()].index.tolist()
    total = len(missing_indices)
    n_batches = int(np.ceil(total / batch_size))
    print(f"{total} samples")
    print(f"{n_batches} batches of size {batch_size}")
    print("batch ", end=" ")
    
    for ii,batch_indices in enumerate(chunked(missing_indices, batch_size)):
        print(ii, end=" ")
        batch_df = df.loc[batch_indices]
        prompt_series = batch_df[description_column] + " " + batch_df[question_column]
        prompts = prompt_series.tolist()
        results = get_batch_llm_responses(prompts, model_id=model_id, token=token)
        df.loc[batch_indices, answer_column] = results
    print()

# Classify Examples

In [ ]:
for column in (ANSWER_COLUMN, REASONING_COLUMN):
    if column not in classification_df:
        classification_df[column] = pd.Series(np.nan, dtype="string")
    
classify(
    df=classification_df,
    description_column=DESCRIPTION_COLUMN,
    question_column=QUESTION_COLUMN,
    answer_column=ANSWER_COLUMN,
    model_id=MODEL_ID,
    token=HF_TOKEN
)

# Save Results

In [ ]:
with pd.option_context("display.max_colwidth", None):
    display(
        classification_df.head(5).style.set_properties(**{"text-align": "left"})
    )
    
GiveMeSomeCredit.save_dataframe(
    classification_df, "classification_prompts.csv"
)